In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
%run ./transform_and_create_tables.ipynb

<class 'str'>


/tmp/ipykernel_74600/3054090517.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLCONTRATO = base_originacao.VLCONTRATO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
/tmp/ipykernel_74600/3054090517.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLPARCELA = base_originacao.VLPARCELA.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
/tmp/ipykernel_74600/3054090517.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_carteira.VLCONTRATO =

contém 0 registros duplicados na base_carteira!
contém 0 registros duplicados na base_originacao!


In [3]:
conn = sqlite3.connect('teste_ailos.db')

base_originacao = pd.read_sql_query("SELECT * FROM base_originacao", conn)
base_carteira = pd.read_sql_query("SELECT * FROM base_carteira", conn)

In [4]:
%reload_ext sql
%sql sqlite:///teste_ailos.db

# RESOLUÇÃO DO EXERCÍCIO EM SQL
Identifique o volume de contratos (por cooperativa) que estão em 31/jul/2021 e que **não estão** em 31/ago/2021. Utilize como chaves: COOPERATIVA, CPF, CONTA, CONTRATO, DSPRODUTO e VLCONTRATO

In [5]:
%%sql
with jul_contracts as (
    select 
        DTBASE
        ,COOPERATIVA
        ,CPF
        ,CONTA
        ,CONTRATO
        ,DSPRODUTO
        ,VLCONTRATO 
    from 
        base_carteira
    where
        date(DTBASE) = '2021-07-31'
),

ago_contracts as (
    select 
        DTBASE
        ,COOPERATIVA
        ,CPF
        ,CONTA
        ,CONTRATO
        ,DSPRODUTO
        ,VLCONTRATO 
    from 
        base_carteira
    where
        date(DTBASE) = '2021-08-31'
)

select 
    jul_contracts.COOPERATIVA
    ,count(distinct jul_contracts.CONTRATO) as Qnt
from 
    jul_contracts
left join 
    ago_contracts
        on jul_contracts.COOPERATIVA = ago_contracts.COOPERATIVA
        and jul_contracts.CPF = ago_contracts.CPF
        and jul_contracts.CONTA = ago_contracts.CONTA
        and jul_contracts.CONTRATO = ago_contracts.CONTRATO
        and jul_contracts.DSPRODUTO = ago_contracts.DSPRODUTO
        and jul_contracts.VLCONTRATO = ago_contracts.VLCONTRATO
where 
    ago_contracts.CONTRATO is null
group by 
    jul_contracts.COOPERATIVA;

 * sqlite:///teste_ailos.db
Done.


COOPERATIVA,Qnt
A,299
B,5
C,10
D,14
E,25


# RESOLUÇÃO DO EXERCÍCIO EM PYTHON
Identifique o volume de contratos (por cooperativa) que estão em 31/jul/2021 e que **não estão** em 31/ago/2021. Utilize como chaves: COOPERATIVA, CPF, CONTA, CONTRATO, DSPRODUTO e VLCONTRATO

In [6]:
# criando os dataframes filtrados
jul_contracts = base_carteira[base_carteira.DTBASE == '2021-07-31 00:00:00']
ago_contracts = base_carteira[base_carteira.DTBASE == '2021-08-31 00:00:00']

In [7]:
ago_contracts.head()

,index,DTBASE,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,SDDEVEDOR,QTDIASATRASO,SDATRASO,VLPROVISAO
22470,22470,2021-08-31 00:00:00,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,8802.6,0.0,0.0,30.6
22471,22471,2021-08-31 00:00:00,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,3702.6,0.0,0.0,20.4
22472,22472,2021-08-31 00:00:00,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,4732.8,0.0,0.0,30.6
22473,22473,2021-08-31 00:00:00,A,512,217818,223696,Empréstimo,2020-01-31 00:00:00,3111.0,2121.6,0.0,0.0,10.2
22474,22474,2021-08-31 00:00:00,A,1142,126440,498576,Financiamento,2021-04-05 00:00:00,106681.8,107018.4,0.0,0.0,316.2


In [8]:
# mergeando os dois dataframes pelas chaves indicadas
check_df = jul_contracts.merge(ago_contracts, how='left', on=['COOPERATIVA', 'CPF', 'CONTA', 'CONTRATO', 'DSPRODUTO', 'VLCONTRATO'])

In [9]:
# alterando o check_df para conter só os valores que existem em Jul mas não existem em Ago
check_df = check_df[check_df.index_y.isnull() == True]

In [10]:
check_df.groupby('COOPERATIVA')['CONTRATO'].count()

COOPERATIVA
A    299
B      5
C     10
D     14
E     25
Name: CONTRATO, dtype: int64